In [55]:
import cv2
import mediapipe as mp
import time
import numpy as np
import pandas as pd

In [56]:
# only run this once ever per session
# try:
#     whole = pd.read_csv("signs.csv")

# except:
#     print("No dataset found")

In [57]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

In [58]:
cap = cv2.VideoCapture(0)
ls = []
elapsed = 0

with mp_hands.Hands(
    model_complexity = 0,
    min_detection_confidence = 0.5,
    min_tracking_confidence = 0.5
) as hands:
    
    while cap.isOpened():

        time_start = time.time()

        retval, frame = cap.read()

        if not retval:
            print("Camera Error; Exiting")
            break

        frame.flags.writeable = False
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(frame)
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:

                mp_drawing.draw_landmarks(
                    frame,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style()
                )

        if results.multi_hand_world_landmarks:
            for hand_landmarks in results.multi_hand_world_landmarks:
                ls.append([[lmk.x, lmk.y, lmk.z] for lmk in hand_landmarks.landmark])


        fps = 1 / (time.time() - time_start)

        frame = cv2.putText(frame, f"FPS: {round(fps)}", (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 3)

        cv2.imshow("Hand Landmarks", frame)

        if cv2.waitKey(1) == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

I0000 00:00:1736799653.717286  163587 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1736799653.718166  165146 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 22.3.6), renderer: Mesa Intel(R) HD Graphics 630 (KBL GT2)
W0000 00:00:1736799653.732278  165140 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1736799653.736259  165142 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
QObject::moveToThread: Current thread (0x3fc1c90) is not the object's thread (0x49727f0).
Cannot move to target thread (0x3fc1c90)

QObject::moveToThread: Current thread (0x3fc1c90) is not the object's thread (0x49727f0).
Cannot move to target thread (0x3fc1c90)

QObject::moveToThread: Current thread (0x3fc1c90) is not the object's thread (0x49727f0).
Cannot move to target

In [59]:
# normalize coordinates to be relative centered at landmark 0
ls = np.array(ls)

for l in ls:
    for i in range(21):
        l[i][0] = l[i][0] - l[0][0]
        l[i][1] = l[i][1] - l[0][1]

ls

array([[[ 0.        ,  0.        ,  0.02373048],
        [ 0.04066961, -0.01043457,  0.02180493],
        [ 0.0239715 , -0.02262115,  0.02071799],
        ...,
        [-0.00501768,  0.01542006,  0.02196643],
        [-0.00088255,  0.0163014 ,  0.03289159],
        [ 0.00753338,  0.01970478,  0.03604242]],

       [[ 0.        ,  0.        ,  0.04482082],
        [ 0.01332199, -0.0373792 ,  0.02450195],
        [ 0.02681769, -0.02556468,  0.0093732 ],
        ...,
        [ 0.00074313,  0.02038035,  0.02706593],
        [ 0.01132854,  0.02454362,  0.03446177],
        [ 0.01659727,  0.02055635,  0.0376773 ]],

       [[ 0.        ,  0.        ,  0.03441314],
        [ 0.0103036 , -0.04691191,  0.01814886],
        [ 0.01790629, -0.02936722,  0.0102737 ],
        ...,
        [ 0.00955733,  0.01590415,  0.02413366],
        [ 0.0133451 ,  0.03026509,  0.0231602 ],
        [ 0.01221293,  0.03821246,  0.02447397]],

       ...,

       [[ 0.        ,  0.        ,  0.04751744],
        [ 0

In [60]:
reshaped_ls = ls.copy()
reshaped_ls = reshaped_ls.reshape(-1, 63)
reshaped_ls.shape

(695, 63)

In [61]:
df_columns = []
for i in range(21):
    df_columns.append(f"Landmark {i} x")
    df_columns.append(f"Landmark {i} y")
    df_columns.append(f"Landmark {i} z")

In [62]:
# create dataframe where rows are frames and columns are coordinates of all 21 landmarks (x, y have distinct columns)
df = pd.DataFrame(reshaped_ls, columns=df_columns)
df.shape

(695, 63)

In [63]:
# add label for handpose (DO NOT USE EMOJIS LOL)
df["Label"] = "okay"

In [64]:
try:
    whole = pd.concat([whole, df], axis=0)
except:
    whole = df

whole.shape

(1865, 64)

In [65]:
whole["Label"].value_counts()

Label
okay        695
not good    669
good        501
Name: count, dtype: int64

In [66]:
whole[whole["Label"] == "good"].head()

,Landmark 0 x,Landmark 0 y,Landmark 0 z,Landmark 1 x,Landmark 1 y,Landmark 1 z,Landmark 2 x,Landmark 2 y,Landmark 2 z,Landmark 3 x,...,Landmark 18 x,Landmark 18 y,Landmark 18 z,Landmark 19 x,Landmark 19 y,Landmark 19 z,Landmark 20 x,Landmark 20 y,Landmark 20 z,Label
0,0.0,0.0,0.074142,0.005651,-0.023243,0.054495,0.015459,-0.036240,0.032941,0.020159,...,0.009361,0.027842,0.013516,0.021177,0.025828,0.032920,0.008301,0.027068,0.043717,good
1,0.0,0.0,0.078138,0.006657,-0.022482,0.056573,0.015234,-0.038615,0.041245,0.018452,...,0.015356,0.031457,0.010221,0.024944,0.027643,0.030496,0.008958,0.027493,0.038383,good
2,0.0,0.0,0.079869,0.002561,-0.022894,0.056903,0.010248,-0.039731,0.041357,0.013721,...,0.014184,0.033699,0.014535,0.023454,0.029137,0.032537,0.007882,0.028423,0.036945,good
3,0.0,0.0,0.079102,-0.001550,-0.023990,0.055559,0.007605,-0.041376,0.039932,0.012197,...,0.015668,0.032852,0.015608,0.022845,0.027410,0.033287,0.007575,0.027552,0.039873,good
4,0.0,0.0,0.075820,-0.001731,-0.019855,0.052006,0.006349,-0.037924,0.038831,0.011998,...,0.017386,0.033109,0.015334,0.021472,0.027903,0.030481,0.007091,0.028001,0.036172,good


In [67]:
# just in case of bad data
def delete_label(df, label):
    return df[df["Label"] != label]

# whole = delete_label(whole, "not good")
# whole["Label"].value_counts()

In [69]:
whole.to_csv("new_signs_en.csv", index=False)